# Chest Pneumonia Classification in Keras

This notebook mirrors the TensorFlow/Keras workflow authored by **bhavya1600**. It performs exploratory analysis, builds a convolutional neural network, and evaluates the model on the validation and test splits.

In [ ]:
import itertools
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
from tensorflow.keras.preprocessing.image import ImageDataGenerator

DATA_ROOT = Path('..') / 'data' / 'chest_xray'
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 42

## Data generators

We rely on `ImageDataGenerator` for on-the-fly augmentation and rescaling. The generators are pointed at the `train`, `val`, and `test` directories following the shared repository layout.

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_test_datagen = ImageDataGenerator(rescale=1.0 / 255)

train_generator = train_datagen.flow_from_directory(
    DATA_ROOT / 'train',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    seed=SEED
)

val_generator = val_test_datagen.flow_from_directory(
    DATA_ROOT / 'val',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

test_generator = val_test_datagen.flow_from_directory(
    DATA_ROOT / 'test',
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False
)

## Visualize a batch

Having a quick look at the augmented images ensures the orientation and class balance look reasonable.

In [ ]:
images, labels = next(train_generator)
plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(images[i])
    plt.title('PNEUMONIA' if labels[i] else 'NORMAL')
    plt.axis('off')
plt.tight_layout()
plt.show()

## Build the CNN

The architecture combines convolutional blocks with batch normalization and dropout to reduce overfitting.

In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=IMG_SIZE + (3,)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D((2, 2)),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

## Train the network

The callbacks monitor validation performance and reduce the learning rate on plateau.

In [ ]:
callbacks = [
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
]

history = model.fit(
    train_generator,
    epochs=25,
    validation_data=val_generator,
    callbacks=callbacks
)

## Plot learning curves

A combined plot of accuracy and loss helps benchmark the run against previous experiments.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
axes[0].plot(history.history['accuracy'], label='train')
axes[0].plot(history.history['val_accuracy'], label='val')
axes[0].set_title('Accuracy')
axes[0].legend()

axes[1].plot(history.history['loss'], label='train')
axes[1].plot(history.history['val_loss'], label='val')
axes[1].set_title('Loss')
axes[1].legend()
plt.show()

## Evaluate on the test split

We gather predictions, compute classification metrics, and visualize the confusion matrix.

In [ ]:
preds = model.predict(test_generator)
pred_labels = (preds.ravel() > 0.5).astype(int)
true_labels = test_generator.classes

print(classification_report(true_labels, pred_labels, target_names=test_generator.class_indices.keys()))

cm = confusion_matrix(true_labels, pred_labels)
plt.figure(figsize=(5, 5))
plt.imshow(cm, cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(len(test_generator.class_indices))
plt.xticks(tick_marks, test_generator.class_indices.keys(), rotation=45)
plt.yticks(tick_marks, test_generator.class_indices.keys())

for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, cm[i, j], horizontalalignment='center', color='white' if cm[i, j] > cm.max() / 2 else 'black')

plt.ylabel('True label')
plt.xlabel('Predicted label')
plt.tight_layout()
plt.show()

## Save the model

Persisting the weights allows the trained network to be re-used or shared without rerunning the entire training process.

In [ ]:
model.save('bhavya1600_pneumonia_classifier.h5')